In [1]:
import gin
import sys 
import os

sys.path.append("../")
from utils import   utils

In [2]:
gin.parse_config_file('../config/base_config.gin')
utils.set_env_variables()

In [3]:
data_root = os.environ["data_root"]
render_data_name = os.environ["render_data_name"]


In [4]:
def depth_to_pc(depth, K, TWO):
    pass

def visualize_pc():
    pass

from pathlib import Path
from PIL import Image
import json
import numpy as np
from torch.utils.data import Dataset

class base_scene_dataset(Dataset):

    def __init__(self, mode = "train", split_txt = None, num_samples = -1, color = False):
        mode = mode.lower()
        if mode =='validation': mode = 'val'
        assert mode in ["train", "val", "test"], "invalid mode, select train, val, or test"
        self.data_root = Path(os.environ["data_root"])
        self.render_data_name = Path(os.environ["render_data_name"])
        self.render_data_path = self.data_root / self.render_data_name
        self.color = color
        if split_txt is None:
            self.collect_scenes()
        else:
            NotImplementedError

    def replace_suffix(self, path_name ,add, remove= '.depth.png'):
        l = int(-1*len(remove))
        path_name = path_name[:l] + add
        return path_name
    
    def check_exists (self, paths_list):
        exists = True
        for path in paths_list:
            if not Path(path).exists():
                exists = False
                print(f"Warning {path} does not exist and the scene will be dropped")
        return exists

    def collect_scenes(self):
        depth_path_gen = self.render_data_path.rglob('*.depth.png')
        self.depth_path, self.camera_path, self.obj_info_path, self.seg_path = [], [], [], []
        if self.color:
            self.color_path = []
        for depth in depth_path_gen:
            depth_path = str(depth)
            depth_path = depth_path
            camera_path = self.replace_suffix(depth_path, '.camera_data.json')
            obj_info_path = self.replace_suffix(depth_path, '.object_datas.json')
            seg_path = self.replace_suffix(depth_path, '.segmentation.png')
            paths = [depth_path, camera_path, obj_info_path, seg_path]

            if self.color:
                color_path = self.replace_suffix(depth_path, '.rgb.png')
                paths.append(color_path)
            
            paths_exist = self.check_exists(paths)

            if paths_exist:
                self.depth_path.append(depth_path)
                self.camera_path.append(camera_path)
                self.obj_info_path.append(obj_info_path)
                self.seg_path.append(seg_path)
                if self.color:
                    self.color_path.append(color_path)
                    
    def __getitem__(self, idx):
        cam_file = open(self.camera_path[idx])
        obj_info_file = open(self.obj_info_path[idx])
        return_dict = {
        "depth" : np.asarray(Image.open(self.depth_path[idx])),
        "camera" : json.load(cam_file) ,
        "obj_info" : json.load(obj_info_file)  ,
        "seg" : np.asarray(Image.open(self.seg_path[idx])) ,
        }

        if self.color:
            return_dict.update({
                "color" : np.asarray(Image.open(self.color_path[idx])) 
            })
            
        return return_dict
    
    def __len__ (self):
        return len(self.depth_path)



In [19]:
scenes = base_scene_dataset()
scene["obj_info"][0]

{'label': 'shapenet_04460130_92e4cc97df1ebe141203ec4ca6ccf208',
 'TWO': [[0.10432620006604738,
   -0.06876918927786269,
   0.9882948619972277,
   -0.08752231990422647],
  [0.11888591295540152, 0.18749665005271288, 0.01746787903200947]],
 'bbox_amodal': [642, 402, 716, 454],
 'bbox_modal': [642, 402, 714, 454],
 'visib_fract': 0.9457959880495092,
 'unique_id': 1}

In [33]:
scene = scenes[0]

seg = scene["seg"]
seg_mask = seg == scene["obj_info"][0]["unique_id"]

xy = np.indices(scene["camera"]["resolution"])
depth = scene["depth"]
xy_z = np.concatenate([xy,depth[np.newaxis]], axis=0)
xy_z_masked = xy_z[:,seg_mask]
K = scene["camera"]["K"]

XYZ = xy_z_masked 
XYZ[0] = XYZ[0] * XYZ[-1]
XYZ[1] = XYZ[1] * XYZ[-1]

XYZ[0,:] = (XYZ[0,:] - K[0][-1])/K[0][0] 
XYZ[1,:] = (XYZ[1,:] - K[1][-1])/K[1][1] 

XYZ = XYZ/100


In [27]:
XYZ.max(1) - XYZ.min(1)

array([0.601, 0.889, 2.972])

In [891]:
np.abs(XYZ.max(1)) - np.abs(XYZ.min(1))

array([ 60.1,  88.9, 297.2])

In [34]:
np.abs(XYZ.max(1) -XYZ.min(1)) 

array([ 6.01,  8.89, 29.72])

In [893]:
np.abs(XYZ.min(1))

array([ 26.6,  40.8, 139.2])

In [894]:
Q, T_WC = scene["camera"]["TWC"]
RWC = utils.quaternion_rotation_matrix(Q)

xyz = RWC.T @ XYZ
xyz =  xyz - np.asarray(T_WC)[:,np.newaxis]

Q, TWO = scene["obj_info"][0]["TWO"]
RWO = utils.quaternion_rotation_matrix(Q)
xyz = xyz + np.asarray(TWO)[:,np.newaxis]

xyz = RWO @ xyz
  
print(xyz.max(1),
      xyz.min(1))

[314.62766743 312.37612755 -43.16578847] [ 100.32217318  100.52512145 -138.63619909]


In [56]:
import open3d as o3d

QWC, tWC = scene["camera"]["TWC"]
QWO, tWO = scene["obj_info"][0]["TWO"]
RWC = utils.quaternion_rotation_matrix(QWC)
RWO = utils.quaternion_rotation_matrix(QWO)

TWC, TWO = np.eye(4), np.eye(4)
TWC[:3,:3] = RWC
TWC[:3,-1] = tWC

TWO[:3,:3] = RWO
TWO[:3,-1] = tWO
XYZ_ = np.array(
    [[-1,0,0],
     [0,-1,0],
     [0,0,1]]
) @ XYZ
TCO = np.linalg.inv(TWC) @ TWO
#yy = TCO[:3,:3].T @ XYZ_
#yy = -1 * TCO[:3,:3].T @ TCO[:3,-1:] + yy

mesh_ = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(XYZ.T))

In [57]:
import open3d as o3d
path = Path(os.environ["data_root"]) / scene["obj_info"][0]["label"].replace("_","/") / "models" / "model_normalized.obj"

mesh = o3d.io.read_triangle_mesh(str(path))

yy = TCO[:3,:3] @ (np.asarray((mesh.vertices * diag  + centroid))).T
yy = yy + TCO[:3,-1:]

mesh.vertices = o3d.utility.Vector3dVector(yy.T)


[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

In [58]:
o3d.visualization.draw_geometries([mesh, mesh_] )

In [21]:
import json
file = open(Path(os.environ["data_root"]) / scene["obj_info"][0]["label"].replace("_","/") / "models" / "model_normalized.json")

ff = json.load(file)

In [22]:
diag = np.linalg.norm(np.array(ff['max']) - np.array(ff['min']))
centroid = np.asarray(ff['centroid'])

In [23]:
diag



44.912662252017526

In [38]:
yy.T.max(0)-yy.T.min(0)

array([31.34450505, 22.66550058, 29.53755434])

In [13]:
XYZ_.max(1) - XYZ_.min(1)

array([ 60.1,  88.9, 297.2])

In [28]:
np.array(ff['max']) - np.array(ff['min'])

array([17.79063, 38.1845 , 15.57513])

In [36]:
yy.T.min(0)

array([-0.34966151, -0.18617377, -0.32652084])

In [48]:
XYZ_.max(1)

array([  8.67,  -4.08, -13.92])

In [49]:
XYZ_.min(1)

array([  2.66, -12.97, -43.64])

In [42]:
yy.max(1)

array([15.64027591, 14.30394071, 14.8726343 ])

In [43]:
yy.min(1)

array([-15.70422914,  -8.36155986, -14.66492004])

In [60]:
print(XYZ.max(1), "\n", XYZ.min(1))

[ 8.67 12.97 43.64] 
 [ 2.66  4.08 13.92]


In [61]:
print(XYZ_.max(1), "\n", XYZ_.min(1))

[-2.66 -4.08 43.64] 
 [ -8.67 -12.97  13.92]


In [63]:
import open3d as o3d
path = Path(os.environ["data_root"]) / scene["obj_info"][0]["label"].replace("_","/") / "models" / "model_normalized.obj"

mesh = o3d.io.read_triangle_mesh(str(path))

x = mesh.vertices * diag + centroid

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

In [68]:
print(x.max(0), "\n", x.min(0))

[12.19838667 38.18450237  3.19421053] 
 [-5.59223761e+00 -1.94360967e-05 -1.23809171e+01]


In [69]:
x.shape

(71011, 3)

In [70]:
TWO

array([[-0.96877369, -0.11766673,  0.21824777,  0.11888591],
       [-0.15419021,  0.97522138, -0.15864686,  0.18749665],
       [-0.19417242, -0.18734457, -0.96291177,  0.01746788],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [80]:
(TCO[:3,:3].T @ x.T).max(1)

array([1.4842383 , 6.79944807, 9.60068656])

In [81]:
(TCO[:3,:3].T @ x.T).min(1)

array([-36.3631888 , -13.29135153,  -6.44919524])